### In this notebook we identify the top Manhattan destinations and the breakdown of people residence

In [1]:
import zipfile 
import pysal as ps
%matplotlib inline
import numpy as np
import ast
import json
import urllib
import urllib2
from cStringIO import StringIO
import pandas as pd
import requests
import os
import geopandas as gpd
import matplotlib.pylab as pl
from fiona.crs import from_epsg
import matplotlib.pylab as pl
from matplotlib import colors
import shapely
import seaborn as sns
import gzip
import mapreduce as mr
import csv

In [56]:
trackind= gpd.GeoDataFrame.from_file('Data/fortopdestinations.geojson')

In [57]:
trackind.head()

3000100  3002100  3019100  3024100  3025300  3025500  3025700  3025901  \
0      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1      1.0      NaN      NaN      NaN      1.0      NaN      NaN      NaN   
2      1.0      NaN      NaN      NaN      NaN      1.0      NaN      NaN   
3      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4      2.0      NaN      NaN      NaN      NaN      1.0      1.0      NaN   

   3025902  3026100 ...  4060300  4061301  4061302  4062300  4062500  4062700  \
0      NaN      NaN ...      NaN      NaN      NaN      NaN      NaN      NaN   
1      NaN      1.0 ...      NaN      NaN      NaN      NaN      NaN      NaN   
2      NaN      1.0 ...      NaN      NaN      NaN      NaN      NaN      NaN   
3      NaN      NaN ...      NaN      NaN      NaN      1.0      NaN      NaN   
4      NaN      NaN ...      NaN      NaN      NaN      NaN      NaN      NaN   

   4062900  boro_ct_2010                                           geometry  \
0      NaN       1000100  (POLYGON ((-74.0438776163991 40.69018767537123...   
1      NaN       1000201  (POLYGON ((-73.98449647076764 40.7095096639394...   
2      NaN       1000202  (POLYGON ((-73.98237340572007 40.7055433504376...   
3      NaN       1000500  (POLYGON ((-74.01092841268032 40.6844914725429...   
4      NaN       1000600  (POLYGON ((-73.99022012603876 40.7144049398979...   

   id  
0   0  
1   1  
2   2  
3   3  
4   4  

[5 rows x 282 columns]

In [58]:
trackind.columns

Index([     u'3000100',      u'3002100',      u'3019100',      u'3024100',
            u'3025300',      u'3025500',      u'3025700',      u'3025901',
            u'3025902',      u'3026100',
       ...
            u'4060300',      u'4061301',      u'4061302',      u'4062300',
            u'4062500',      u'4062700',      u'4062900', u'boro_ct_2010',
           u'geometry',           u'id'],
      dtype='object', length=282)

In [59]:
reduced = gpd.GeoDataFrame.from_file('Data/Reducedtrack.geojson')

In [60]:
reduced.head()

Within15min  Within30min  bigManhattanCommute boro_code boro_ct_2010  \
0            1            1                    1         3      3028501   
1            1            1                    0         3      3036300   
2            1            1                    1         3      3036502   
3            1            1                    0         3      3039700   
4            1            1                    1         3      3039900   

  boro_name cdeligibil ct_2010 ctlabel  \
0  Brooklyn          I  028501  285.01   
1  Brooklyn          E  036300     363   
2  Brooklyn          E  036502  365.02   
3  Brooklyn          E  039700     397   
4  Brooklyn          E  039900     399   

                                            geometry   id ntacode  \
0  (POLYGON ((-73.93856854782675 40.6988481355494...  114    BK78   
1  (POLYGON ((-73.91400482503951 40.6732394879431...  133    BK79   
2  (POLYGON ((-73.90267831840295 40.6757897287538...  134    BK79   
3  (POLYGON ((-73.91464246723457 40.6875070419845...  137    BK78   
4  (POLYGON ((-73.92115385566102 40.6927966934759...  138    BK78   

          ntaname  puma     shape_area     shape_leng  
0  Bushwick South  4002   895739.69679  4167.68973824  
1      Ocean Hill  4007  2785736.31011  8445.46688322  
2      Ocean Hill  4007  1252061.56461  6132.19606711  
3  Bushwick South  4002  1712820.27346  6560.68169372  
4  Bushwick South  4002  1627809.30279  6391.34887911

In [67]:
selectedtracks=[]


selectedtracks = list(reduced[reduced.bigManhattanCommute==True].boro_ct_2010)
selectedtracks.append('boro_ct_2010')
selectedtracks.append('geometry')

In [68]:
trackind[selectedtracks]

3028501  3036502  3039900  3040100  3041300  3041500  3041700  3041900  \
0        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
2        NaN      NaN      NaN      NaN      NaN      NaN      1.0      2.0   
3        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4        NaN      NaN      NaN      4.0      3.0      NaN      1.0      NaN   
5        1.0      2.0     15.0     11.0     18.0     12.0     11.0      6.0   
6        NaN      NaN      1.0      1.0      NaN      NaN      NaN      1.0   
7        4.0      6.0     22.0     20.0     11.0      9.0      8.0     13.0   
8        NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
9        1.0      3.0      NaN      1.0      1.0      NaN      1.0      NaN   
10       NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
11       2.0      4.0     17.0     17.0      7.0      9.0     11.0     11.0   
12       NaN      NaN      1.0      NaN      NaN      1.0      NaN      NaN   
13       NaN      NaN      NaN      2.0      3.0      NaN      1.0      NaN   
14       1.0      NaN      1.0      5.0      2.0      1.0      3.0      6.0   
15       3.0      5.0      7.0      8.0     10.0     17.0      5.0     11.0   
16       1.0      1.0      NaN      NaN      1.0      1.0      3.0      NaN   
17       2.0      NaN      4.0      3.0      2.0      1.0      1.0      3.0   
18       NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
19       2.0     12.0     21.0     32.0     20.0     24.0     15.0     18.0   
20       NaN      1.0      NaN      1.0      NaN      2.0      NaN      NaN   
21       NaN      NaN      NaN      NaN      NaN      NaN      NaN      1.0   
22       NaN      NaN      1.0      NaN      NaN      NaN      NaN      NaN   
23       NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
24       NaN      NaN      NaN      1.0      NaN      NaN      NaN      NaN   
25       NaN      NaN      1.0      NaN      NaN      2.0      NaN      NaN   
26       NaN      1.0      NaN      NaN      NaN      NaN      NaN      NaN   
27       NaN      1.0      NaN      NaN      1.0      NaN      NaN      1.0   
28       2.0     10.0     23.0     19.0     26.0     29.0     27.0     24.0   
29       4.0      1.0      4.0     10.0      2.0      NaN      1.0      1.0   
..       ...      ...      ...      ...      ...      ...      ...      ...   
255      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
256      NaN      NaN      NaN      NaN      2.0      NaN      3.0      2.0   
257      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
258      NaN      NaN      1.0      NaN      NaN      NaN      NaN      NaN   
259      NaN      NaN      1.0      NaN      NaN      NaN      NaN      NaN   
260      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
261      NaN      NaN      NaN      NaN      NaN      1.0      NaN      NaN   
262      NaN      NaN      NaN      NaN      NaN      NaN      1.0      NaN   
263      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
264      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
265      NaN      NaN      NaN      NaN      1.0      NaN      NaN      NaN   
266      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
267      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
268      NaN      NaN      NaN      1.0      NaN      NaN      NaN      NaN   
269      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
270      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
271      NaN      NaN      NaN      NaN      NaN      NaN      NaN      1.0   
272      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
273      NaN      NaN      NaN      NaN      NaN      N

In [69]:
count = trackind.iloc[:,:-3].sum(1)

### Top 10 Manhattan Destinations from affected Borough

In [70]:
count.sort_values(ascending=False)[:10]


28     5884.0
33     5767.0
74     5038.0
19     4908.0
107    4457.0
31     4452.0
7      4056.0
5      3571.0
120    3545.0
113    3455.0
dtype: float64

In [71]:
mask = count.sort_values(ascending=False)[:10].index

In [72]:
trackind.loc[mask]

3000100  3002100  3019100  3024100  3025300  3025500  3025700  3025901  \
28      30.0     21.0     14.0     16.0     23.0     32.0     11.0     15.0   
33      32.0     40.0     25.0     19.0     23.0     36.0     11.0     11.0   
74       7.0     12.0     13.0      9.0     25.0     24.0     11.0     12.0   
19      21.0     27.0     17.0      8.0     14.0     31.0     15.0     16.0   
107     31.0     31.0     38.0     12.0     12.0     25.0      7.0      9.0   
31      32.0      6.0     20.0      6.0     19.0     34.0     15.0      9.0   
7       34.0     48.0     24.0     23.0     11.0     44.0     12.0     13.0   
5       45.0     45.0     20.0      8.0     13.0     13.0      6.0      NaN   
120     37.0     74.0     14.0     15.0     10.0     10.0      8.0     14.0   
113     19.0     26.0      9.0      5.0     14.0     19.0     18.0      1.0   

     3025902  3026100 ...   4060300  4061301  4061302  4062300  4062500  \
28      26.0     41.0 ...      15.0     30.0      NaN     12.0     14.0   
33      22.0     27.0 ...      12.0     25.0      1.0     26.0     21.0   
74      18.0     19.0 ...       5.0     17.0      NaN      9.0      9.0   
19      33.0     34.0 ...       9.0     26.0      NaN      7.0     14.0   
107     16.0     33.0 ...       4.0     18.0      2.0      6.0      4.0   
31       9.0     23.0 ...      11.0     29.0      3.0     12.0     12.0   
7       11.0     18.0 ...       3.0     13.0      NaN      9.0      5.0   
5        8.0     20.0 ...       8.0     33.0      NaN     10.0     12.0   
120      7.0     23.0 ...       8.0     13.0      2.0      3.0      8.0   
113      7.0     19.0 ...       9.0     25.0      4.0      8.0      3.0   

     4062700  4062900  boro_ct_2010  \
28       9.0     22.0       1002900   
33      18.0     18.0       1003300   
74       6.0     10.0       1007600   
19       6.0     11.0       1002100   
107      8.0     11.0       1010900   
31      12.0     21.0       1003100   
7        2.0      5.0       1000900   
5       11.0      8.0       1000700   
120     13.0     11.0       1011900   
113     14.0      7.0       1011300   

                                              geometry   id  
28   (POLYGON ((-73.99605872744372 40.7162316407135...   28  
33   (POLYGON ((-74.00530986129975 40.7218241918130...   33  
74   (POLYGON ((-73.98407636095357 40.7491021253839...   74  
19   (POLYGON ((-74.00920540087833 40.7161879531960...   19  
107  (POLYGON ((-73.98594653087758 40.7523168055555...  107  
31   (POLYGON ((-73.99995620864657 40.7180170950871...   31  
7    (POLYGON ((-74.00078295341706 40.6942865156646...    7  
5    (POLYGON ((-73.99799957540132 40.6987976855442...    5  
120  (POLYGON ((-73.98225528672133 40.7573873648954...  120  
113  (POLYGON ((-73.98411754823195 40.7548420534646...  113  

[10 rows x 282 columns]

In [74]:
trackind.loc[28].shape

(282L,)

In [76]:
trackind.loc[28].dropna().shape

(278L,)

In [81]:
trackind.loc[28].dropna().index[:-3][0]

u'3000100'

In [83]:
trackind.loc[28].dropna().values[:-3][0]

30.0

In [106]:
theDict={}
for i in mask:
    theDict[trackind.loc[i,'boro_ct_2010']] = {}
    for j in range(len(trackind.loc[i].dropna().index[:-3])):
        theDict[trackind.loc[i,'boro_ct_2010']][trackind.loc[i].dropna().index[:-3][j]]=trackind.loc[i].dropna().values[:-3][j]
    #theDict[trackind.loc[i,'boro_ct_2010']]['geometry']=trackind.loc[i,'geometry']

In [99]:
theDict

{u'1000700': {u'3000100': 45.0,
  u'3002100': 45.0,
  u'3019100': 20.0,
  u'3024100': 8.0,
  u'3025300': 13.0,
  u'3025500': 13.0,
  u'3025700': 6.0,
  u'3025902': 8.0,
  u'3026100': 20.0,
  u'3026300': 6.0,
  u'3027100': 7.0,
  u'3027700': 14.0,
  u'3027900': 14.0,
  u'3028100': 21.0,
  u'3028300': 14.0,
  u'3028501': 1.0,
  u'3028502': 10.0,
  u'3028700': 9.0,
  u'3028900': 8.0,
  u'3029100': 10.0,
  u'3029300': 9.0,
  u'3029500': 19.0,
  u'3029700': 14.0,
  u'3029900': 7.0,
  u'3030100': 8.0,
  u'3030300': 15.0,
  u'3030700': 15.0,
  u'3030900': 6.0,
  u'3034500': 8.0,
  u'3034700': 15.0,
  u'3034900': 22.0,
  u'3035500': 16.0,
  u'3035700': 2.0,
  u'3035900': 26.0,
  u'3036100': 10.0,
  u'3036300': 11.0,
  u'3036501': 10.0,
  u'3036502': 2.0,
  u'3036700': 4.0,
  u'3036900': 14.0,
  u'3037100': 9.0,
  u'3037300': 14.0,
  u'3037500': 9.0,
  u'3037700': 17.0,
  u'3037900': 17.0,
  u'3038100': 12.0,
  u'3038300': 15.0,
  u'3038500': 19.0,
  u'3038700': 20.0,
  u'3038900': 13.0,
  u'30

In [108]:
with open('Data/Top10DestManhattan.json', 'w') as fp:
    json.dump(theDict, fp)